In [2]:
import re
import os
from pathlib import Path

In [118]:
mat_properties_re = re.compile(
    r'''^
    (\w+)                               # argument
    (?:[ \t]*\(([\s\d,\:]+)\))?         # size
    (?:[ \t]*(\w+))?                    # argument class
    (?:[ \t]*\{(.+)\})?                 # validators
    (?:[ \t]*\=[ \t]*
        (\w+(?:\([\w\s\{\}\@'",]*\))?)  # value / function and optional arguments
    )?                                  
    (?:\s+\#+[ \t]*(.*))?               # Comment
    $''', re.VERBOSE
)

mat_validators_re_pat = re.compile(
    r'''
    [ \t]*
        (\w+(?:\([\w\s\{\}\@'",]*\))?)
    [ \t]*,?
    ''', re.VERBOSE
)

def parse_property(prompt:str):
    (arg, size, arg_class, validators, default, comment) = mat_properties_re.match(prompt.strip()).groups()
    if size:
        size = [dim.strip() for dim in size.split(',')]
    validator_list = mat_validators_re_pat.findall(validators) if validators else None
    return (arg, size, arg_class, validator_list, default, comment)

parse_property("arg2 (1,:) double {mustBeImData(Data), mustBeInRange(Data,0,255), mustBeMember(Interp,{'linear','cubic','spline'})} = uint8(0)  #This is")

In [4]:
mat_ext_sig_re = re.compile(
    r'''^((?:[^/]*/)*)?                 # Path
        ((?:[@+]\w+\.)*)                # Namespace
        (\w+)                           # Function name
        $''', re.VERBOSE
)

mat_namespace_re_pat = re.compile(
    r'([@+]\w+)\.',                     # Namespace list
    re.VERBOSE        
)

def parse_autodoc_input(prompt:str, src_path:str):

    (path, namespacePath, filename) = mat_ext_sig_re.match(prompt.strip()).groups()
    namespace = mat_namespace_re_pat.findall(namespacePath)

    basepath = Path(src_path)
    abspath = (basepath / Path(path or '.')).resolve()
    filepath = (abspath / '/'.join(namespace)).resolve()

    return (filepath, filename, abspath, namespace)

arg = '../@package.+main.test'
src_path = 'C:/Users'

parse_autodoc_input(arg, src_path)

(WindowsPath('C:/@package/+main'),
 'test',
 WindowsPath('C:/'),
 ['@package', '+main'])